## Capstone 1: Analyzing Pitch Sequence Data in Baseball from Retrosheets.org

### $\S$5.4 Data Wrangling

*submitted by Brian Camp*

This project is designed around event data for MLB Baseball games that can be downloaded from **https://www.retrosheet.org**. At that site, there are also some tools provided that help with the preprocessing some of the data.

In this project, data will from 2010 through 2019 will be processed. The goal of this part of the project is to produce a pandas DataFrame (or several) that will combine all of the desired data from the years 2010 through 2018. As a brief preview, there are 30 event files for each year. Each event file has approximately 6000 separate events (all of the plate appearances that happened in that location). So in each year there are about $30\times6000 \approx 180000$ separate events. Finally for the nine years to be considered this means that there will be around 1.8 million rows of data (each row representing a different event in a baseball game).

**Retrosheets.org**<br>
Data is given as event files named either <code>filename.evn</code> or <code>filename.eva</code>. These files are processed using the utility <code>bevent.exe</code> which is command line utility in Windows. The general format of an event file is given as <code>yyyyloc.eva</code> where the **yyyy** stands of the year, the **loc** stands for the home team (or home stadium) and the ending (either <code>.eva</code> or <code>.evn</code> stands for American League or National League respectively.

From **https://www.retrosheet.org/notice.txt**
<pre>
Recipients of Retrosheet data are free to make any desired use of
the information, including (but not limited to) selling it,
giving it away, or producing a commercial product based upon the
data.  Retrosheet has one requirement for any such transfer of
data or product development, which is that the following
statement must appear prominently:

     The information used here was obtained free of
     charge from and is copyrighted by Retrosheet.  Interested
     parties may contact Retrosheet at "www.retrosheet.org".

Retrosheet makes no guarantees of accuracy for the information 
that is supplied. Much effort is expended to make our website 
as correct as possible, but Retrosheet shall not be held 
responsible for any consequences arising from the use the 
material presented here. All information is subject to corrections 
as additional data are received. We are grateful to anyone who
discovers discrepancies and we appreciate learning of the details.
</pre>

**Importing packages to be used in processing or otherwise**

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import itertools as itr
import collections as coll
import re
import os
import subprocess
import pickle

Here the base directory is created. This will allow for sub-directories for the retrosheet data and the resulting processed data to be stored.

In [2]:
basedir = os.getcwd()

The <code>subprocess.run()</code> function will allow the <code>*.exe</code> files to be run from within python. This will also allow for the commands to be run multiple times within a loop. This is necessary as the will be 270 event files to be processed (30 event files per year for 9 years worth of data).

By using the <code>os.getcwd()</code> command and the <code>subprocess.run()</code> command, this will also make this process repeatable for other years if it is desired without too much modification required in the code.

In [5]:
os.chdir(basedir)
print('This will run in the background command window where the Jupyter kernel is running.')

subprocess.run(['retro/bevent.exe','-h'])

This will run in the background command window where the Jupyter kernel is running.


CompletedProcess(args=['retro/bevent.exe', '-h'], returncode=1)

**Creating .csv files: retro_to_csv() function**

The function <code>retro_to_csv()</code> function is created. Its purpose is to run the <code>bevent.exe</code> command on one <code>.eva</code> or <code>.evn</code> event file and output a <code>.csv</code> file.
    
It is important to note that this command should be run in the same directory where the <code>bevent.exe</code> file is. In this case, the retrosheet.org data and the <code>bevent.exe</code> file will be kept in the <code>retro</code> subdirectory. Later, when the command is run in a loop, the <code>.csv</code> output files will be put in the <code>data</code> subdirectory.

In [16]:
def retro_to_csv(fields, teamname, 
                 startdate = '', enddate = '', 
                 outputfile = '', 
                 output = 1, verbose = 1):
    # teamnane in format "yyyyloc.eva or .evn"
    # yyyy is the year, startdate: '-s mmdd', enddate: '-e mmdd'
    # loc is the stadium location
    # eva for american league stadium, evn for national league stadium
    batchname = 'bevent.exe'
    year = ' '.join(['-y',teamname[:4]])

    if outputfile == '':
        outputfile = ''.join([teamname[:7],'.csv']);
        
    strlist = [batchname, year, startdate, enddate, fields, teamname, '>', outputfile]
    batch_str = ' '.join(strlist)

    if verbose:
        print(batch_str)
        
    if output:
        subprocess.run(batch_str, shell=True)
        
    return outputfile

**Initializing the data folder**

Here all <code>.csv</code> files that may be in the data folder are erased. This will prevent any errors when the <code>bevent.exe</code> file is run
using <code>subprocess.run()</code>.

The command <code>os.listdir()</code> returns a list of files in the current working directory which we loop over in this case. If the last four characters of a filename are **.csv** then that file is deleted using <code>os.remove()</code>.

In [17]:
os.chdir(basedir)
os.chdir('data')
for eachfile in os.listdir():
    if eachfile[-4:]=='.csv':
        os.remove(eachfile)

#print(os.getcwd())
os.listdir()

['.ipynb_checkpoints']

**Creating lists and dictionaries by year**

A list of the years is created first. Then a dictionary is created with a list of events for each year. This provides for efficient processing by year of the event files. If the output DataFrame turns out to be too large, this will also allow for creating DataFrames by each year and also allowing for easy processing of teams within a given year based upon the dictionary if needed. The effort here is to make the process somewhat modular so that flexibility is in place if later processing requires it.

$\triangleright$ A list of year names.

In [18]:
yearlist = [str(each) for each in range(2010,2019)]
print(yearlist)

['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']


$\triangleright$ Creating the dictionary of events by year.

In [19]:
os.chdir(basedir)
os.chdir('retro')

retro_event_list = os.listdir()
fields = '-f 0-17,26-51,58-61'

events = coll.defaultdict(list)

for yr in yearlist:
    events[yr]= [team for team in retro_event_list 
                 if team[:4]==yr and (team[-4:].lower()=='.eva' or team[-4:].lower()=='.evn')]

print('For example, the keys of the dictionary are the list of years.')
print(events.keys(),end='\n\n')

print('And for the year 2012 we have:\n')
print(events['2012'])

For example, the keys of the dictionary are the list of years.
dict_keys(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'])

And for the year 2012 we have:

['2012ANA.EVA', '2012ARI.EVN', '2012ATL.EVN', '2012BAL.EVA', '2012BOS.EVA', '2012CHA.EVA', '2012CHN.EVN', '2012CIN.EVN', '2012CLE.EVA', '2012COL.EVN', '2012DET.EVA', '2012HOU.EVN', '2012KCA.EVA', '2012LAN.EVN', '2012MIA.EVN', '2012MIL.EVN', '2012MIN.EVA', '2012NYA.EVA', '2012NYN.EVN', '2012OAK.EVA', '2012PHI.EVN', '2012PIT.EVN', '2012SDN.EVN', '2012SEA.EVA', '2012SFN.EVN', '2012SLN.EVN', '2012TBA.EVA', '2012TEX.EVA', '2012TOR.EVA', '2012WAS.EVN']


**Looping to create the <code>.csv</code> files** 

During the process, the names of the <code>.csv</code> files are stored in a dictionary by the year in which they occur.

In [24]:
%%time
os.chdir(basedir)
os.chdir('retro')

# these are the fields that will be chosen for each event
# (i.e which columns to select) ... more on their meaning below
fields = '-f 0-17,26-51,58-61' # no spaces

for each in os.listdir():
    
    league = each[-3:].lower()
    year = each[:4]
    
    if (league == 'eva' or league == 'evn') and year in yearlist:
        
        outfile = ''.join(['../data/',each.replace('.','_').lower(),'.csv'])
        retro_to_csv(fields, each, outputfile=outfile, verbose=0)

Wall time: 27.1 s


**Creating DataFrames**

The <code>fields</code> variable described which columns were to be used by the <code>bevent.exe</code> command. Below is a list of names representing the information in each column. This list will be used to create the headings in the subsequent data frames. Also, since each <code>.csv</code> file will have the same column headings, using <code>pd.merge()</code> to combine the .csv files will be straightforward.

$\triangleright$ The <code>fields</code> variable - list of column names.

In [25]:
# columns imported: names
# columns 0-17,26-51,58-61  # no spaces when inputting below
colnames = ['gameid', 'awayteam', 'inning', 'teamatbat', 'outs', #0-4
              'balls', 'strikes', 'pitches', 'scoreaway', 'scorehome', #5-9
              'batter', 'batterhand', 'resbatter', 'resbatterhand', 'pitcher', #10-14
              'pitcherhand', 'respitcher', 'respitcherhand', #15-17
              'runner01', 'runner02', 'runner03', 'eventtext', 'leadoffflag', #26-30
              'pinchhitflag', 'batterpos', 'lineupposition', 'eventtype', 'plateappoverflag', #31-35
              'abflag', 'hitvalue', 'SHflag', 'SFflag', 'outsresult', #36-40
              'doubleplayflag', 'tripleplayflag', 'rbiresult', 'wildflag', 'passedflag', #41-45
              'fieldedby', 'battedballtype', 'buntflag', 'foulflag', 'hitlocation', #46-50
              'numberrors', 'destbatter', 'dest01', 'dest02', 'dest03'] #51, 58-61
len(colnames)

48

$\triangleright$ looping to create large dataframe

The smaller DataFrames are created using <code>pd.read_csv()</code>. Then the larger DataFrame is constructed using <code>pd.concat()</code>.

In [26]:
%%time
os.chdir(basedir)
os.chdir('data')

# initializing the dataframe
try:
    del data
except:
    print('"data" variable not present.')
    
data = pd.DataFrame(data=None, columns=colnames)

# now creating the individual dataframes and combining then using pd.concat
for each in os.listdir():
    
    if each[-4:]=='.csv':
        
        #print('processing: ',each)
        df = pd.read_csv(each, names=colnames)
        data = pd.concat([data, df], ignore_index=True)

        
print(data.shape)
print(data.columns)

"data" variable not present.
(1717976, 48)
Index(['gameid', 'awayteam', 'inning', 'teamatbat', 'outs', 'balls', 'strikes',
       'pitches', 'scoreaway', 'scorehome', 'batter', 'batterhand',
       'resbatter', 'resbatterhand', 'pitcher', 'pitcherhand', 'respitcher',
       'respitcherhand', 'runner01', 'runner02', 'runner03', 'eventtext',
       'leadoffflag', 'pinchhitflag', 'batterpos', 'lineupposition',
       'eventtype', 'plateappoverflag', 'abflag', 'hitvalue', 'SHflag',
       'SFflag', 'outsresult', 'doubleplayflag', 'tripleplayflag', 'rbiresult',
       'wildflag', 'passedflag', 'fieldedby', 'battedballtype', 'buntflag',
       'foulflag', 'hitlocation', 'numberrors', 'destbatter', 'dest01',
       'dest02', 'dest03'],
      dtype='object')
Wall time: 4min 57s


**An overview of the DataFrame.**

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1717976 entries, 0 to 1717975
Data columns (total 48 columns):
gameid              object
awayteam            object
inning              object
teamatbat           object
outs                object
balls               object
strikes             object
pitches             object
scoreaway           object
scorehome           object
batter              object
batterhand          object
resbatter           object
resbatterhand       object
pitcher             object
pitcherhand         object
respitcher          object
respitcherhand      object
runner01            object
runner02            object
runner03            object
eventtext           object
leadoffflag         object
pinchhitflag        object
batterpos           object
lineupposition      object
eventtype           object
plateappoverflag    object
abflag              object
hitvalue            object
SHflag              object
SFflag              object
outsresult          obj

In [29]:
data.head()

,gameid,awayteam,inning,teamatbat,outs,balls,strikes,pitches,scoreaway,scorehome,...,fieldedby,battedballtype,buntflag,foulflag,hitlocation,numberrors,destbatter,dest01,dest02,dest03
0,ANA201004050,MIN,1,0,0,2,2,CSBFFBFC,0,0,...,0,NaN,F,F,NaN,0,0,0,0,0
1,ANA201004050,MIN,1,0,1,0,1,FX,0,0,...,4,G,F,F,NaN,0,0,0,0,0
2,ANA201004050,MIN,1,0,2,1,1,CBX,0,0,...,4,G,F,F,NaN,0,0,0,0,0
3,ANA201004050,MIN,1,1,0,3,2,BBCCFBFFFB,0,0,...,0,NaN,F,F,NaN,0,1,0,0,0
4,ANA201004050,MIN,1,1,0,0,1,CX,0,0,...,8,F,F,F,NaN,0,0,1,0,0


**Pickling the DataFrame for later use**

In [28]:
%%time
os.chdir(basedir)
os.chdir('data')

data.to_pickle('retro_mlb_dataframe.pickle')

Wall time: 4.17 s
